In [1]:
import os
from ultralytics import YOLO
from ultralytics.nn.modules.conv import *
from ultralytics.nn.modules.block import *
from ultralytics.data.utils import check_det_dataset, img2label_paths
from ultralytics.engine.results import Boxes, Results
from ultralytics.utils.ops import xywh2xyxy, xyxy2xywhn, xywhn2xyxy
import torch
from PIL import Image
import numpy as np
import shutil
import cv2

In [2]:
def process_quadrant_label(x_min, y_min, x_max, y_max, label, ori_imgsz, new_imgsz):
    class_id, x_center, y_center, width, height = label
    x1, y1, x2, y2 = xywhn2xyxy(np.array([x_center, y_center, width, height]), w=ori_imgsz[1], h=ori_imgsz[0])
    # print(x_min, y_min, x_max, y_max, ori_imgsz, new_imgsz)
    # print(label)
    x1 = max(x1, x_min) - x_min
    y1 = max(y1, y_min) - y_min
    x2 = min(x2, x_max) - x_min
    y2 = min(y2, y_max) - y_min
    x_center, y_center, width, height = xyxy2xywhn(np.array([x1, y1, x2, y2]), w=new_imgsz[1], h=new_imgsz[0])
    return [class_id, x_center, y_center, width, height]

def process_image_label(img_path, label_path, overlap=200):
    img = Image.open(img_path)
    img = np.array(img)
    h, w = img.shape[:2]
    label = np.loadtxt(label_path).reshape(-1, 5)
    mid_x, mid_y = w // 2, h // 2

    # Tính lại vùng ảnh có overlap
    parts = {
        "Top-left": img[:mid_y + overlap, :mid_x + overlap],
        "Top-right": img[:mid_y + overlap, max(mid_x - overlap, 0):],
        "Bottom-left": img[max(mid_y - overlap, 0):, :mid_x + overlap],
        "Bottom-right": img[max(mid_y - overlap, 0):, max(mid_x - overlap, 0):]
    }

    labels_by_part = {"Top-left": [], "Top-right": [], "Bottom-left": [], "Bottom-right": []}
    for lbl in label:
        class_id, x_center, y_center, width, height = lbl
        x_center_px = x_center * w
        y_center_px = y_center * h

        if x_center_px < min(mid_x + overlap, w) and y_center_px < min(mid_y + overlap, h):
            part = "Top-left"
            tmp = process_quadrant_label(
                0,
                0,
                min(mid_x + overlap, w),
                min(mid_y + overlap, h),
                lbl,
                img.shape[:2],
                parts[part].shape[:2]
            )
            labels_by_part[part].append(tmp)
        if x_center_px >= max(mid_x - overlap, 0) and y_center_px < min(mid_y + overlap, h):
            part = "Top-right"
            tmp = process_quadrant_label(
                max(mid_x - overlap, 0),
                0,
                w,
                min(mid_y + overlap, h),
                lbl,
                img.shape[:2],
                parts[part].shape[:2]
            )
            labels_by_part[part].append(tmp)
        if x_center_px < min(mid_x + overlap, w) and y_center_px >= max(mid_y - overlap, 0):
            part = "Bottom-left"
            tmp = process_quadrant_label(
                0,
                max(mid_y - overlap, 0),
                min(mid_x + overlap, w),
                h,
                lbl,
                img.shape[:2],
                parts[part].shape[:2]
            )
            labels_by_part[part].append(tmp)
        if x_center_px >= max(mid_x - overlap, 0) and y_center_px >= max(mid_y - overlap, 0):
            part = "Bottom-right"
            tmp = process_quadrant_label(
                max(mid_x - overlap, 0),
                max(mid_y - overlap, 0),
                w,
                h,
                lbl,
                img.shape[:2],
                parts[part].shape[:2]
            )
            labels_by_part[part].append(tmp)

    return parts, labels_by_part

def crop2quadrant(src_folder, tgt_folder):
    img_folder_path, label_folder_path = os.path.join(src_folder, 'images'), os.path.join(src_folder, 'labels')
    tgt_img_folder_path, tgt_label_folder_path = os.path.join(tgt_folder, 'images'), os.path.join(tgt_folder, 'labels')
    os.makedirs(tgt_img_folder_path, exist_ok=True)
    os.makedirs(tgt_label_folder_path, exist_ok=True)

    for item in os.listdir(img_folder_path):
        img_path = os.path.join(img_folder_path, item)
        # label_path = os.path.join(src_folder, os.path.basename(item).split('.')[0] + '.txt')
        label_path = img2label_paths([img_path])[0]
        part, labels_by_part = process_image_label(img_path, label_path)
        name = item.split('.')[0]
        cnt = 0
        for key, value in part.items():
            img = Image.fromarray(value)
            img.save(os.path.join(tgt_img_folder_path, name + '_' + str(cnt) + '.jpg'), format='JPEG')
            if len(labels_by_part[key]) == 0:
                # Nếu rỗng, tạo một file txt trống
                open(os.path.join(tgt_label_folder_path, name + '_' + str(cnt) + '.txt'), 'w').close()
            else:
                np.savetxt(
                    os.path.join(tgt_label_folder_path, name + '_' + str(cnt) + '.txt'),
                    labels_by_part[key],
                    fmt='%d %0.6f %0.6f %0.6f %0.6f'
                )
            cnt += 1
        
        # break

    return 

# crop2quadrant('datasets/final_malaria_full_class/test', 'datasets/final_malaria_full_class_quadrant/test')

In [3]:
crop2quadrant('datasets/v2_malaria_PA7_5_class/test', 'datasets/v2_malaria_PA7_5_class_quadrant/test')
crop2quadrant('datasets/v2_malaria_PA7_5_class/train', 'datasets/v2_malaria_PA7_5_class_quadrant/train')
crop2quadrant('datasets/v2_malaria_PA7_5_class/val', 'datasets/v2_malaria_PA7_5_class_quadrant/val')
shutil.copy2('datasets/v2_malaria_PA7_5_class/data.yaml', 'datasets/v2_malaria_PA7_5_class_quadrant/data.yaml')

'datasets/v2_malaria_PA7_5_class_quadrant/data.yaml'

## Plot to test

In [4]:

def draw_gt(path, data_yaml_path):
    image = Image.open(path)
    image_np = np.array(image).astype(np.uint8)
    image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

    data_yaml = check_det_dataset(data_yaml_path)
    
    img_height, img_width = image_np.shape[:2]
    label_path = img2label_paths([path.replace("/", "\\")])[0]
    boxes_data = []
    with open(label_path, 'r') as f:
        # Đọc từng dòng
        for line in f:
            # Tách các giá trị trong dòng (theo khoảng trắng)
            values = line.split()
            
            # Lấy class_id, x_center, y_center, width, height
            class_id = int(values[0])
            x_center = float(values[1])
            y_center = float(values[2])
            width = float(values[3])
            height = float(values[4])

            x_min = int((x_center - width / 2) * img_width)
            y_min = int((y_center - height / 2) * img_height)
            x_max = int((x_center + width / 2) * img_width)
            y_max = int((y_center + height / 2) * img_height)
            box_tensor = torch.tensor([x_min, y_min, x_max, y_max, 1.0, class_id])
            boxes_data.append(box_tensor)

    boxes_data = torch.stack(boxes_data)
    # print(boxes_data.ndim)
    # boxes = Boxes(boxes_data, (img_height, img_width))
    results = Results(image_np, path, data_yaml['names'], boxes=boxes_data)

    # for result in results:
    results.save_crop(save_dir=os.path.join('groundtruth', os.path.basename(path)), file_name="detection")
    results.save(os.path.join('groundtruth', os.path.basename(path), "annotated_image.jpg"), conf=False, line_width=4)
    return



In [5]:
# path='datasets/final_malaria_full_class_quadrant/test/images/050_0.tif'
# data_yaml_path = 'datasets/final_malaria_full_class_quadrant/data.yaml'
path='datasets/v2_malaria_PA7_5_class_quadrant/test/images/051Overlay002_3.jpg'
# path='datasets/v2_malaria_PA7_5_class/test/images/051Overlay002.jpg'
data_yaml_path = 'datasets/v2_malaria_PA7_5_class_quadrant/data.yaml'
draw_gt(path, data_yaml_path)